In [1]:
importall POMDPs
using Distributions # for Normal
srand(1);

In [2]:
type LightDark1D <: POMDPs.POMDP{Float64,Int,Int}
    discount_factor::Float64
    correct_r::Float64
    incorrect_r::Float64
    step_size::Int
    movement_cost::Float64
end
LightDark1D() = LightDark1D(0.9, 10, -10, 1, 0)
discount(p::LightDark1D) = p.discount_factor
isterminal(::LightDark1D, s::Float64) = isnan(s);

In [3]:
noise(x) = ceil(Int, abs(x - 5)/sqrt(2) + 1e-2)

function generate_o(p::LightDark1D, s::Float64, a::Int, sp::Float64, rng::AbstractRNG)
    if isnan(sp)
        return 0
    else
        n = noise(sp)
        return round(Int, sp) + rand(rng, -n:n)
    end
end

function generate_s(p::LightDark1D, s::Float64, a::Int, rng::AbstractRNG)
    if a == 0
        return NaN
    else
        return s+a
    end
end

function reward(p::LightDark1D, s::Float64, a::Int, sp::Float64)
    if a == 0
        if abs(s) < 1
            return p.correct_r
        else
            return p.incorrect_r
        end
    else
        return 0.0
    end 
end;

In [4]:


function initial_state_distribution(pomdp::LightDark1D)
    return Normal(2.0, 3.0)
end;

In [4]:
actions(::LightDark1D) = [-1, 0, 1] # Left Stop Right
n_actions(::LightDark1D) = 3

n_actions (generic function with 1 method)

In [1]:
using BasicPOMCP
using POMDPToolbox

In [4]:
solver = POMCPSolver(tree_queries=10000, c=10)
pomdp = LightDark1D()
planner = solve(solver, pomdp);

In [5]:
b = initial_state_distribution(pomdp)
a = action(planner, b)
println("""
    POMCP Recommends action $a for belief $b.

    (this may be a bad choice because the POMCP Parameters are set to their defaults.)
""")

LoadError: [91mMethodError: no method matching initial_state_distribution(::LightDark1D)[39m

In [ ]:
for (s,a,r,sp,o) in stepthrough(pomdp, planner, "sarspo")
    @show (s,a,r,sp,o)
end